# Example A.2

The code below finds the optimal transactions' cost for any pair of channel deposits $l_A$ and $l_B$ when: $C=1$, $D=2$, $\alpha = 0.999$ and payments satisfy $\mathbb{P}[\Delta = 1] = \mathbb{P}[\Delta = -1] = \tfrac{1}{2}$. 

In [113]:
import numpy as np
from scipy.optimize import root

C = 1
D = 2
alpha = 0.999

# By Theorem 3.2 and Remark 3.3 we need to find the fixed point of the following function
def F(v, l_A, l_B):
    N = l_A + l_B 
    base_index = l_A
    
    Fv = np.zeros(N + 1)

    # Compute the vector F(v)
    Fv[0] = alpha * (0.5 * min(C + v[0], C + D + v[base_index]) + 0.5 * min(C + v[0], D + v[base_index], v[1]))
    
    for i in range(1, N):
        Fv[i] = alpha * (0.5 * min(C + v[i], D + v[base_index], v[i - 1]) + 0.5 * min(C + v[i], D + v[base_index], v[i + 1]))
    
    Fv[N] = alpha * (0.5 * min(C + v[0], C + D + v[base_index]) + 0.5 * min(C + v[0], D + v[base_index], v[N - 1]))
    
    return Fv

# The unique fixed point of F is the unique root of the function G(v)=F(v)-v
def G(v, l_A, l_B):
    Fv = F(v, l_A, l_B)
    Gv = Fv - v
    return Gv


# This finds the unique root of the function G, and hence the unique fixed point of F
def T(l_A, l_B, method='hybr'):
    
    # Set up the initial guess vector U-shaped
    N = l_A + l_B + 1
    base_index = l_A
    initial_guess_vector = np.concatenate((base_index - np.arange(base_index), np.arange(N - base_index))) 
    
    # Use the root function from scipy.optimize with flexibility in the method 
    Fixed_Point = root(G, initial_guess_vector, args=(l_A, l_B), method=method)
        
    return Fixed_Point.x[base_index]

In order to find the optimal deposits, we need to minimize the following function over $l_A, l_B \in \mathbb{N}$. This is because payments are of magnitude $1$.

In [115]:
# This is the value function 
def V(l_A, l_B):
    return l_A + l_B + T(l_A, l_B)

V(10,10)

39.653309465173876

When $l_A = l_B = 10$, the value function is 39.653309465173876 < 40. Therefore, it is enough to run through the values $l_A, l_B \in \mathbb{N}$ such that $l_A + l_B \leq 40$. The following code does this and returns the optimal deposits.

In [117]:
min_value = float('inf')
min_l_A = None
min_l_B = None

for l_A in range(1, 41):
    for l_B in range(1, 41 - l_A):
        if l_A + l_B <= 40:
            result = V(l_A, l_B)
            if result < min_value:
                min_value = result
                min_l_A = l_A
                min_l_B = l_B

min_l_A, min_l_B

(12, 13)

Therefore, $l_A^* = 12$ and $l_B^* = 13$ are the optimal channel deposits. In particular, these deposits result in a smaller cost than $l_A = l_B = 12$ or $l_A = l_B = 13$.

In [119]:
V(12, 13), V(12, 12), V(13, 13)

(37.483728714451246, 37.548730853756666, 37.496687296815466)